# DAQHandler

Single-task high-level interface with DAQHandler.

In [ ]:
# Check nidaqmx availability
try:
    import nidaqmx
except ImportError:
    raise RuntimeError(
        "nidaqmx is not installed. Install with: pip install nidaqmx"
    )

In [ ]:
import time
import numpy as np
from nidaqwrapper import DAQHandler, AITask, AOTask, DITask, DOTask, list_devices, list_tasks

# List connected hardware
devices = list_devices()
print(f"Found {len(devices)} device(s):")
for i, dev in enumerate(devices):
    print(f"  Device {i}: {dev['name']} ({dev['product_type']})")

# List saved NI MAX tasks
tasks = list_tasks()
if tasks:
    print(f"\nSaved tasks: {tasks}")
else:
    print("\nNo tasks saved in NI MAX.")

## NI MAX Task Names

Configure DAQHandler with NI MAX task name strings.

In [ ]:
# Edit these to match your NI MAX task names
INPUT_TASK = 'MyInputTask'
OUTPUT_TASK = 'MyOutputTask'

handler = DAQHandler(task_in=INPUT_TASK, task_out=OUTPUT_TASK)
handler.connect()

print(f"Device info: {handler.get_device_info()}")
print(f"Input channels: {handler.get_channel_names()}")
print(f"Sample rate: {handler.get_sample_rate()} Hz")

handler.disconnect()

## Programmatic Tasks

Create AITask/AOTask objects and pass to configure().

In [ ]:
# Edit device_ind and channel_ind to match your hardware
device_ind = 0  # Device 0 from list_devices()
channel_ind = [0, 1]  # AI channels 0 and 1

ai_task = AITask(
    task_name='programmatic_input',
    sample_rate=10000,
    device_ind=device_ind
)
ai_task.add_channel(
    channel_name='accel_x',
    channel_ind=channel_ind[0],
    max_val=10.0,
    min_val=-10.0,
    units='V'
)
ai_task.add_channel(
    channel_name='accel_y',
    channel_ind=channel_ind[1],
    max_val=10.0,
    min_val=-10.0,
    units='V'
)

handler = DAQHandler()
handler.configure(task_in=ai_task)
handler.connect()

print(f"Configured with {handler.get_channel_names()}")

handler.disconnect()

## Reading Data

Single sample with read(), buffer drain with read_all_available().

In [ ]:
# Single sample read
handler = DAQHandler(task_in=INPUT_TASK)
handler.connect()

single_sample = handler.read()
print(f"Single sample shape: {single_sample.shape}")
print(f"Values: {single_sample}")

handler.disconnect()

In [ ]:
# Read all available data from buffer
handler = DAQHandler(task_in=INPUT_TASK)
handler.connect()

# Sleep to let buffer fill
time.sleep(0.5)

data = handler.read_all_available()
print(f"Buffer data shape: {data.shape}")
print(f"First 5 samples:\n{data[:5]}")

handler.disconnect()

## Writing Data

Single DC value with write().

In [ ]:
handler = DAQHandler(task_out=OUTPUT_TASK)
handler.connect()

# Write single DC value (2.5V)
handler.write(2.5)
time.sleep(1.0)

# Write zero
handler.write(0.0)

handler.disconnect()

## Continuous Generation

Generate continuous waveforms with generate() and stop_generation().

In [ ]:
handler = DAQHandler(task_out=OUTPUT_TASK)
handler.connect()

# Generate 1 Hz sine wave
sample_rate = handler.get_device_info()['output']['sample_rate']
duration = 1.0  # seconds
n_samples = int(sample_rate * duration)
t = np.linspace(0, duration, n_samples)
sine_wave = 2.0 * np.sin(2 * np.pi * 1.0 * t)

# Reshape for single channel: (n_samples, 1)
sine_wave = sine_wave.reshape(-1, 1)

handler.generate(sine_wave)
time.sleep(3.0)  # Let it run for 3 seconds

handler.stop_generation()
handler.disconnect()

## Digital I/O

Read and write digital lines with DITask/DOTask.

In [ ]:
# Edit device and lines to match your hardware
device_name = 'Dev1'

# Create digital input task
di = DITask(task_name='digital_inputs')
di.add_channel(channel_name='switches', lines=f'{device_name}/port0/line0:3')
di.start()

# Create digital output task
do = DOTask(task_name='digital_outputs')
do.add_channel(channel_name='leds', lines=f'{device_name}/port1/line0:3')
do.start()

# Configure handler with digital tasks
handler = DAQHandler()
handler.configure(task_digital_in=di, task_digital_out=do)
handler.connect()

# Write digital outputs (4 lines: [True, False, True, False])
handler.write_digital([True, False, True, False])
time.sleep(0.5)

# Read digital inputs
digital_state = handler.read_digital()
print(f"Digital input state: {digital_state}")

handler.disconnect()

## Context Manager

Automatic cleanup with context manager pattern.

In [ ]:
with DAQHandler(task_in=INPUT_TASK) as handler:
    handler.connect()
    data = handler.read()
    print(f"Read data: {data}")
    # Automatic disconnect on exit